<a href="https://colab.research.google.com/github/y-hiroki-radiotech/llm-final-task/blob/main/inference_vllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -U langchain-community langchain-huggingface vllm triton fugashi unidic-lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 k

### プロンプトの作成部分

In [1]:
!git clone https://github.com/y-hiroki-radiotech/llm-final-task.git

fatal: destination path 'llm-final-task' already exists and is not an empty directory.


In [2]:
%cd llm-final-task

/content/llm-final-task


In [13]:
from prompt import PromptStock

prompt_stock = PromptStock()

### jsonファイルの読み込み

In [4]:
import pandas as pd

# JSONLファイルを読み込む
file_path = 'elyza-tasks-100-TV_0.jsonl'
data = pd.read_json(file_path, lines=True)

# DataFrameの表示
data

,task_id,input
0,0,野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。
1,1,花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されていま...
2,2,駅について学ぶ上で知っておくべきポイントは？
3,3,以下の説明を読んだユーザーが感じるであろう疑問は？\n\n水の都ベネチアでは、知られざるエピ...
4,4,観光に最適な列車旅を5つ挙げてください。それぞれの提案には、列車の名前、運行区間、特徴を詳し...
...,...,...
95,95,以下の文章について、筆者がどのような意図でこの文章を書いたか、あなたならどのように解釈します...
96,96,若くして看護師を目指し、緩和ケアの道に進む人たちは患者さんとの向き合い方や感情のコントロール...
97,97,漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。\n\n1....
98,98,次の物語の展開を予想してみましょう。\n\nまる、さんかく、しかくの顔をした3きょうだいがい...


### 新しいデータに対して、タスクのラベルを与える。

In [5]:
from transformers import pipeline

model_name = "hiroki-rad/bert-base-classification-ft"
classify_pipe = pipeline(model=model_name, device="cuda:0")

results: list[dict[str, float | str]] = []
for example in data.itertuples():
    # モデルの予測結果を取得
    model_prediction = classify_pipe(example.input)[0]
    # 正解のラベルIDをラベル名に変換
    results.append( model_prediction["label"])

data["label"] = results
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,task_id,input,label
0,0,野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。,Task_Solution
1,1,花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されていま...,Creative_Generation
2,2,駅について学ぶ上で知っておくべきポイントは？,Knowledge_Explanation
3,3,以下の説明を読んだユーザーが感じるであろう疑問は？\n\n水の都ベネチアでは、知られざるエピ...,Information_Extraction
4,4,観光に最適な列車旅を5つ挙げてください。それぞれの提案には、列車の名前、運行区間、特徴を詳し...,Analytical_Reasoning
...,...,...,...
95,95,以下の文章について、筆者がどのような意図でこの文章を書いたか、あなたならどのように解釈します...,Task_Solution
96,96,若くして看護師を目指し、緩和ケアの道に進む人たちは患者さんとの向き合い方や感情のコントロール...,Task_Solution
97,97,漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。\n\n1....,Information_Extraction
98,98,次の物語の展開を予想してみましょう。\n\nまる、さんかく、しかくの顔をした3きょうだいがい...,Creative_Generation


### モデルのインストール

In [6]:
from langchain_community.llms import VLLM

model_name = "hiroki-rad/llm-jp-llm-jp-3-13b-16-ft"

llm = VLLM(model=model_name,
           quantization="awq")


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

INFO 12-10 14:28:02 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-10 14:28:02 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='hiroki-rad/llm-jp-llm-jp-3-13b-16-ft', speculative_config=None, tokenizer='hiroki-rad/llm-jp-llm-jp-3-13b-16-ft', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=hiroki

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 12-10 14:30:14 model_runner.py:1077] Loading model weights took 25.5730 GB
INFO 12-10 14:30:14 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=26.54GiB peak_torch_memory=26.94GiB memory_usage_post_profile=26.55GiB non_torch_memory=0.55GiB kv_cache_size=8.12GiB gpu_memory_utilization=0.90
INFO 12-10 14:30:15 gpu_executor.py:113] # GPU blocks: 665, # CPU blocks: 327
INFO 12-10 14:30:15 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 2.60x
INFO 12-10 14:30:17 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-10 14:30:17 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

In [8]:
# テンプレート部分
from langchain.prompts import PromptTemplate

template = """
ユーザー: 質問を良く読んで、適切な回答をしてください。
{context}
質問:{input}
回答:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "input"],
    template_format="f-string"
)

In [18]:
vllm_chain = prompt | llm

In [22]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema.runnable import RunnablePassthrough

json_parser = JsonOutputParser()

prompt_string = prompt_stock.get_prompt()

test_chain = (
    {"context": prompt_string, "input": RunnablePassthrough()}
    | vllm_chain
)

TypeError: PromptStock.get_prompt() missing 1 required positional argument: 'keyword'